# Activity 8

* Download the Activity8 lab and upload it onto Google Colab.
* Answer the Activity8 questions in Canvas and populate the cells below
* Submit Activity8 questions on Canvas and upload the PDF version of this lab:
>* To submit the this lab as PDF, go to File, click Print, then save it as PDF instead of printing it

# Business Problem

Henry Hill is the Director of Operations Analytics at a luxury handbag company and has hired you to perform prescriptive analytics to make decisions about three new handbags: Handbag A, Handbag B, and Handbag C.

First, you will use decision analysis to choose which of the 3 handbags to produce based on different decision strategies, with and without probability information. Then, after the success of your initial production choice, you will use linear programming to find the optimal mix of handbags to produce to maximize revenue.

# Import Packages

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 28.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Part 1: Decision Analysis

## Data

You want to determine which handbag (Handbag A, Handbag B, or Handbag C) to sell to generate the most revenue. Handbag A sells for 100, Handbag B sells for 200, and Handbag C sells for 375.

You believe demand for Handbag A will be between 0 and 200 bags, demand for Handbag B will be between 0 and 100 bags, and demand for Handbag C will be between 0 and 50 bags.

You will simulate demand for 100 possible outcomes and use the minimum, average, and maximum values to create a payoff table of the worst case, most likely, and best case demand (and revenue) for each handbag.

In [ ]:
# do not manipluate this cell - just run it

np.random.seed(123)

rand_revenue = pd.DataFrame({
   'A':  np.random.randint(0, 200, 100) * 100,
   'B': np.random.randint(0, 100, 100) * 200,
   'C': np.random.randint(0, 50, 100) * 375})

rand_revenue

,A,B,C
0,10900,1800,4125
1,12600,17400,3750
2,6600,2800,1125
3,9800,16600,16875
4,1700,14000,8625
...,...,...,...
95,9800,12400,7875
96,300,13400,12000
97,13900,2200,14625
98,300,5200,375


In [ ]:
# do not manipluate this cell - just run it

payoff_table = pd.DataFrame({'Worst_Case': rand_revenue.min(),
                             'Most_Likely': rand_revenue.mean(),
                             'Best_Case': rand_revenue.max()})

payoff_table

,Worst_Case,Most_Likely,Best_Case
A,100,9721.00,19900
B,0,10192.00,19800
C,375,9273.75,18375


## Q1

* Run the code cell below to obtain the maximum revenue for each handbag.
* Based on the output, using an optimistic strategy you should manufacture Handbag _____?
* Select the right answer from Canvas. "A"

In [ ]:
# do not manipluate this cell - just run it

payoff_table.max(axis = 1)

,0
A,19900.0
B,19800.0
C,18375.0


## Q2

* Run the code cell below to obtain the minimum revenue for each handbag.
* Based on the output, using an pessimistic strategy you should manufacture Handbag _____?
* Select the right answer from Canvas. "C"

In [ ]:
# do not manipluate this cell - just run it

payoff_table.min(axis = 1)

,0
A,100.0
B,0.0
C,375.0


## Q3

* Run the code cell below to obtain the best payoff for each outcome and create the opportunity loss matrix
* Based on the output, using an opportunity loss strategy you should manufacture Handbag _____?
* Select the right answer from Canvas. "B"

In [ ]:
# do not manipluate this cell - just run it

best_outcomes = payoff_table.max(axis = 0)
print('best outcomes')
print(best_outcomes)
print('')
opp_loss = pd.DataFrame({'Worst_Case': best_outcomes.iloc[0] - payoff_table.iloc[:,0],
                             'Most_Likely': best_outcomes.iloc[1] - payoff_table.iloc[:,1],
                             'Best_Case': best_outcomes.iloc[2] - payoff_table.iloc[:,2]})
print('opportunity loss matrix')
print(opp_loss)
print('')
print('maximum opportunity loss')
print(opp_loss.max(axis = 1))

best outcomes
Worst_Case       375.0
Most_Likely    10192.0
Best_Case      19900.0
dtype: float64

opportunity loss matrix
   Worst_Case  Most_Likely  Best_Case
A       275.0       471.00        0.0
B       375.0         0.00      100.0
C         0.0       918.25     1525.0

maximum opportunity loss
A     471.0
B     375.0
C    1525.0
dtype: float64


## Q4

* Which of the following code lines should you use to output the average payoff for each handbag?
* Select the right answer from Canvas, paste it below, and run the cell

In [ ]:
# Copy and paste your answer from Canvas to Here

print(payoff_table.mean(axis = 1))

A    9907.000000
B    9997.333333
C    9341.250000
dtype: float64


## Q5

* No code needed.
* Select the right answer in Canvas.

## Q6

* Run the code cell below to obtain the expected value payoffs where the worst case outcome occurs with 60% probability, the most likely outcome occurs with 20% probability and the best case outcome occurs with 20% probability.
* Based on the output, using an expected value strategy you should manufacture Handbag _____?
* Select the right answer from Canvas.

In [ ]:
# do not manipluate this cell - just run it

exp_val = pd.DataFrame({'Worst_Case': payoff_table.iloc[:,0] * .6,
                             'Most_Likely': payoff_table.iloc[:,1]* .2,
                             'Best_Case': payoff_table.iloc[:,2]* .2})
print('expected value payoffs')
print(exp_val.sum(axis = 1))

expected value payoffs
A    5984.20
B    5998.40
C    5754.75
dtype: float64


# Part 2: Linear Optimization

## Data

After success with your manufacturing decision using Decision Analysis, Henry decides to produce all three handbags, but needs you to identify the optimal number of each handbag to produce to maximize revenue given production constraints.

Handbag A sells for 100, Handbag B sells for 200, and Handbag C sells for 375.

You are limited in the amount of leather, buttons, and zippers available for production. You can use at most 100 yards of leather at most. Handbag A requires 0.8 yards, Handbag B requires 2 yards, and handbag C requires 4.5 yards. You have 500 buttons available to use. Handbag A uses 7 buttons, Handbag B uses 8 buttons, and Handbag C uses 5 buttons. You have 400 zippers available to use. Handbag A requires 5 zippers, Handbag B requires 8 zippers, and Handbag C requires 7 zippers. Due to capacity limitations, at most, you can produce a total of 150 handbag.

## Q7

* Run the code cell below to set up the linear programming problem.
* Based on the description of the business problem and the output, the constraint on the number of buttons is __________________.
* Select the right answer from Canvas.

In [ ]:
# do not manipluate this cell - just run it

model = LpProblem(name = "max_revenue", sense = LpMaximize)
x = LpVariable(name = "A", lowBound = 0)
y = LpVariable(name = "B", lowBound = 0)
z = LpVariable(name = "C", lowBound = 0)
model += 100 * x + 200 * y + 375 * z
model += (.8 * x + 2 * y + 4.5 * z <= 100, "leather")
model += (7 * x + 8 * y + 5 * z <= 500, "buttons")
model += (5 * x + 8  * y + 7 * z <= 400, "zippers")
model += (x + y + z <= 150, "total_bags")
model

max_revenue:
MAXIMIZE
100*A + 200*B + 375*C + 0
SUBJECT TO
leather: 0.8 A + 2 B + 4.5 C <= 100

buttons: 7 A + 8 B + 5 C <= 500

zippers: 5 A + 8 B + 7 C <= 400

total_bags: A + B + C <= 150

VARIABLES
A Continuous
B Continuous
C Continuous

## Q8

* Which of the following code lines should you use to solve the linear programming problem?
* Select the right answer from Canvas, paste it below, and run the cell

In [ ]:
# Copy and paste your answer from Canvas to Here

model_sol = model.solve()

## Q9

* Run the code cell below to output the optimal values of the decision variables and objective function.
* Based on the output, how many of handbag A should you produce?
* Select the right answer from Canvas.

In [ ]:
# do not manipluate this cell - just run it
print("Total Revenue ", model.objective.value())

for var in model.variables():
    print(var.name, "=", var.varValue)

Total Revenue  10461.956562500001
A = 59.782609
B = 4.076087
C = 9.7826087


## Q10

* No code needed.
* Select the right answer in Canvas.